In [26]:
#import snappy_utils
import os, sys
sys.path.append("../")
import settings
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import zipfile
import time

In [27]:
footprint = settings.footprint
sentinel_api_user = settings.sentinel_api_user
sentinel_api_key = settings.sentinel_api_key
data_path = settings.data_path

In [15]:
class Sentinelsat_products:
    def __init__(self, date_start, date_finish, footprint=footprint, platformname="Sentinel-2"):
        self.date_start = date_start
        self.date_finish = date_finish
        self.platform = platformname
        self.api = SentinelAPI(sentinel_api_user, sentinel_api_key, 'https://scihub.copernicus.eu/dhus')
        self.wkt_footprint = footprint
        self.products = self.query_products(self.date_start, self.date_finish)

    def query_products(self, date_start, date_finish, platformname="Sentinel-2"):
        # search by polygon, time, and Hub query keywords
        products = self.api.query(self.wkt_footprint, area_relation='Contains',
                            date = (self.date_start, self.date_finish),
                            platformname = platformname)
        return products
    
    def filter_products(self, instrument, level, p_type, timeliness):
        removed_products = []
        for product_key in self.products:
            odata = self.api.get_product_odata(product_key, full=True)
            #print(odata)
            if self.platform == "Sentinel-3":
                product_instrument = odata["Instrument"]
                product_level = odata["Product level"]
                product_type = odata["Product type"]
                mission_type = odata["Mission type"]
                product_timeliness = odata["Timeliness Category"]
                #filter only from Level 1 OLCI instrument with NTC full resolution
                conditions = (
                (product_instrument == instrument) and (p_type in product_type) 
                and product_timeliness == timeliness and product_level == level
                            )
            if self.platform == "Sentinel-2":
                product_instrument = odata["Instrument"]
                product_level = odata["Processing level"]            
                conditions = (product_instrument == instrument) and (product_level == level)
                
            if conditions:
                #print(instrument, product_level, product_type)
                pass                
            else:
                removed_products.append(product_key)
        for key in removed_products:
            del self.products[key]

    def download_products(self, make_subset=True, delete_zip=False):
        print("----------")
        for key in self.products:
            file_name = self.products[key]["filename"]
            file_date = self.products[key]["summary"][:16].split("Date: ")[1]
            print(file_date)
            download_path = os.path.join(data_path, file_date)
            if not os.path.exists(download_path):
                os.makedirs(download_path)
            # if it was downloaded before it won't download again            
            download_info = self.api.download(key, directory_path=download_path)
            if download_info["Online"] == False:
                time.sleep(10)
                continue
            zip_path = download_info["path"]
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(download_path)
            if delete_zip:
                os.remove(zip_path)
            if make_subset:
                extracted_directory = os.path.join(download_path, file_name)
                product = snappy_utils.read_product(extracted_directory)
                subset = snappy_utils.make_subset(product, settings.footprint)
                snappy_utils.write_product(subset, os.path.join(download_path, "laguna.tif"), "GeoTiff")
                snappy_utils.write_product(subset, os.path.join(download_path, "laguna.dim"), "BEAM-DIMAP")

### Generic data download code

In [ ]:
date1 = date(2015, 11, 1)
date2 = date(2016, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-2A", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

### Download 2015-2016 data

In [ ]:
date1 = date(2015, 11, 1)
date2 = date(2016, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

### Download 2017-2018 data

In [ ]:
date1 = date(2017, 11, 1)
date2 = date(2018, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

### Download 2018-2019 data

In [ ]:
date1 = date(2018, 11, 1)
date2 = date(2019, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

### Download 2019-2020 data

In [ ]:
date1 = date(2019, 11, 1)
date2 = date(2020, 5, 31)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)

### Download 2020-2021 data

In [ ]:
date1 = date(2020, 11, 1)
date2 = date(2021, 3, 27)

datos = Sentinelsat_products(date1, date2, footprint=footprint)
datos.filter_products(instrument="MSI", level="Level-1C", p_type=None, timeliness=None)
datos.download_products(make_subset=False)